In [1]:
# Project Title: Ride-Share Algorithm Tester 
#
# Author: Richard Kershner
#         with assistance from: ---
#         using code from: ---
#
# Completion Date: 2023_10-08
#
# Algorithms()
#
# Description: 
#        - Holds the processes for creating different routing options. 
#        - 1.  Finds different pick-up and drop-off stop options.
#        - 2.a  Each test algorithm recieves existing route and different options.
#        - 2.b  Each test algorithm returns sorted list of "best" fits of options.
#        - 3.  Returned sorted lists are tested in sorted order for if they are valid.
#
# Precondition:
#        Route class submitted has list of stops with location (x, Y) and time of the stop
#        Request has a pick-up location(X,Y), drop-of location(X,Y) and a requested time
#        A mapper routine is supplied for looking up drive times between two locations
#        Restrictions for the options are hard coded into the class
#            A maximum and minimum time from the requested time the ride can be changed by
#            A maximum time from pick-up time, the drop-off time can be placed.
#
# Postcondition:
#        Provides ordered list of pick-up and drop-off stops for options
#        Provides a list of algorithms being tested and corresponding functions to call 
#        Current algorithms tested:
#            list get_sorted_Time():
#                Returns list of stops ordered by which rides are closest to requested time
#                If times are equal, the latter time is chosen     
#            list get_sorted_TravelTime():
#                Returns list of stops ordered by which one has shortest ride time for request
#                equal ride times goes to the one with pick_up time closest to request
#            get_sorted_Distance():
#                Returns list of stops ordered by which overall route change has lowest distance traveled
#                equal route distances goes to the one with pick_up time closest to request
#            list get_sorted_VectorMinimum():
#                Returns list based on other riders on the shared ride and the change in the vector
#                    vector math is based on grid and not pythagorean math.
#                    Total vector is (X2-X1) + (X3-X2) + ... + (Y2-Y1) + (Y3-Y2)+....
#                    This is different from get_sorted_Distance which is pulled from mapper
#                equal ride times goes to the one with pick_up time closest to request
#        NOTE: ON ALL ALGORITHMS:  drop-off has to be added to route before pick-up
#                if done in reverse, the insert index will NOT work
# Implementation:
#         1.  Options for pick-up stops is generated.
#             a.  Pick-up options must exist in the min/max window of time.
#             b.  Pick-up options are based on before/after fit of existing stops
#         2.  A new list is generated with each pick-up stop having drop-off stops options.
#             a.  Drop-off stops can NOT have down drive time where the passanger sits on the bus.
#             b.  Drop-off times must be within set limit of maximum ride time
#
# include files go here (lines included below just an example)
#        from Mapper import Mapper

class Algorithms():
    def __init__(self, mapper):
        
        # --- initialize variables ---
        self.mapper = mapper
        
        # --- Global Constants ---
        self.MIN_SCHEDULE_WINDOW = -45
        self.MAX_SCHEDULE_WINDOW = 45
        self.MAX_RIDE_TIME = 45
        
    # ================= INDIVIDUAL ALGORITHMS ================
    
    # ------------------------------------------------------------------- sorted time closest to request
    # Author(s):  Richard Kershner
    # Description:
    #        Sorts options based on closest time to requested time
    #        Multiple closest times goes to the time greater then
    #        Lastly, closest to lowest dropoff pnt insert
    def get_sorted_Time(self, routeStops, request):
        options = self.get_options(request[2], stops)
        
        for o in range(len(options)):
            deltaTime = abs(options[o][1] - request[2])
            greaterThen = request[2] > options[o][1]
            options[o].insert(0,greaterThen)
            options[o].insert(0,deltaTime)
            
        # sort (reverse order of importance) by 
        #     closest drop off pointer, closest index, then closest time
        options = sorted(options, key=lambda x:x[5]) # 3 + 2(added rows)
        options = sorted(options, key=lambda x:x[1])
        options = sorted(options, key=lambda x:x[0])
        
        print()
        print(92)
        print("get_sorted_time... pick-up closest to request time")
        print(options)
        print()
        
        # remove sorting columns
        options = [sublist[2:] for sublist in options]
        
        return options
    
    # ------------------------------------------------------------------- sorted travel time all passangers
    # Author(s):  Richard Kershner
    # Description:
    #        Sorts options based on over all travel time for all passangers minimized
    #        Multiple equal options sort on time closest to requested time
    def get_sorted_TravelTime(self, routeStops, request):
        options = self.get_options(request[2], stops)
        
        # for each index (dictionary), who is on the bus
        stopPassangers = dict()
        passangers = []
        for i in range(len(routeStops)):
            # pickup passanger if not onboard
            if routeStops[i][0] not in passangers:
                passangers.append(routeStops[i][0])
                stopPassangers[i] = passangers.copy()
            # dropoff passanger if already on board
            else:
                passangers = list(filter(lambda p: p!= routeStops[i][0], passangers))
            stopPassangers[i] = passangers.copy()
            
        for i in range(len(options)):
            deltaTime = 0
            # for pickup and dropoff
                # calcualte add time passanger count: A -> new stop & new stop -> B
                # calculate remove time for passanger count: A -> new stop -> B, time A -> B
                
            # no rides, no delta time
            if len(routeStops) != 0:
                
                # pickup A is pnt.  pnt + 1 is B.   If pnt + 1 is passed endof stops, nothing
                if options[i][0] < (len(routeStops) - 1):
                    address1 = request[0]
                    addressA = routeStops[options[i][0]][4]
                    addressB = routeStops[options[i][0] + 1][4]
                    deltaTime =+ len(stopPassangers[options[i][0]]) * self.mapper.getRouteInfo(address1, addressA)['time']
                    deltaTime =+ len(stopPassangers[options[i][0]+1]) * self.mapper.getRouteInfo(addressB, address1)['time']
                    

                # drop of  is B.  pnt - 1 is A.  if at start, nothing there, so nothing
                if options[i][2] != 0:
                    address2 = request[1]
                    addressA = routeStops[options[i][0] - 1][4]
                    addressB = routeStops[options[i][0]][4]
                    deltaTime =+ len(stopPassangers[options[i][0]-1]) * self.mapper.getRouteInfo(addressA, address2)['time']
                    deltaTime =+ len(stopPassangers[options[i][0]]) * self.mapper.getRouteInfo(address2, addressB)['time']
            
            options[i].append(deltaTime)
            deltaTime = abs(options[i][1] - request[2])
            options[i].append(deltaTime)
                                                      
        # sort by reverse importance, lowest drop off index, deltaTime from request, ridersTime
        options = sorted(options, key=lambda x:x[2])
        options = sorted(options, key=lambda x:x[5])
        options = sorted(options, key=lambda x:x[4])
        
        print()
        print("160 - over all passanger ride time RAW")
        print(options)
        print()
        
        # remove sorting columns
        options = [sublist[:4] for sublist in options]
        return options
    

    # ------------------------------------------------------------------- sorted distance
    # Author(s):  Richard Kershner
    # Description:
    #        Sorts options based on over all travel distance minimized
    #        Multiple equal options sort on time closest to requested time
    def get_sorted_Distance(self, routeStops, request):
        options = self.get_options(request[2], stops)
        for i in range(len(options)):
            deltaDistance = 0
            
            # ----- calcualte change in distance driven before and after each insert (pick-up and drop-off) ----
            #           note, if insert is same number, drop-off before and pick-up after computes
            #                 as one distance, the distance between pick-up and drop-off
            # if ride exists before pick-up
            if options[i][0] != 0:
                # compute distance prior to pickup.  insert of 1 would push existing 1 forward, previous is i-1
                address1 = routeStops[options[i-1][0]][4]
                address2 = request[0]
                deltaDistance =+self.mapper.getRouteInfo(address1, address2)['distance']
                
            # if ride exists after drop-off
            if options[i][2] < len(routeStops):
                # compute and add distance after drop-off. pushes existing i forward, so next ride is i
                address1 = routeStops[options[i][2]][4]
                address2 = request[0]
                deltaDistance =+self.mapper.getRouteInfo(address1, address2)['distance']
                
            if options[i][0] == options[i][2]:
                # compute and add distance between pick-up and drop-off
                address1 = request[0]
                address2 = request[1]
                deltaDistance =+self.mapper.getRouteInfo(address1, address2)['distance']
                
            # if not same insert slot,meening they are next to each other, then rides exist between
            else:
                # compute and add distance after pick-up and next ride in route
                address1 = request[0]
                address2 = routeStops[options[i][0]][4]
                deltaDistance =+self.mapper.getRouteInfo(address1, address2)['distance']
                # compute and add distance before drop-off and previous ride in route
                address1 = routeStops[options[i][0] - 1 ][4]
                address2 = request[1] 
                deltaDistance =+self.mapper.getRouteInfo(address1, address2)['distance']

            options[i].append(deltaDistance)
            deltaTime = abs(options[i][1] - request[2])
            options[i].append(deltaTime)
        
        # sort by reverse importance, lowest drop off index, deltaTime from request, distance
        options = sorted(options, key=lambda x:x[2])
        options = sorted(options, key=lambda x:x[5])
        options = sorted(options, key=lambda x:x[4])
        print()
        print("222 - vehicle distance test raw")
        print(options)
        print()
        
        # remove sorting columns
        options = [sublist[:4] for sublist in options]
        
        return options
    
    # ------------------------------------------------------------------- sorted drive time
    # Author(s):  Richard Kershner
    # Description:
    #        Sorts options based on over all drive time minimized
    #        Multiple equal options sort on time closest to requested time
    def get_sorted_DriveTime(self, routeStops, request):
        options = self.get_options(request[2], stops)
        for i in range(len(options)):
            deltaTime = 0
            
            # ----- calcualte change in time driven before and after each insert (pick-up and drop-off) ----
            #           note, if insert is same number, drop-off before and pick-up after computes
            #                 as one distance, the distance between pick-up and drop-off
            # if ride exists before pick-up
            if options[i][0] != 0:
                # compute distance prior to pickup.  insert of 1 would push existing 1 forward, previous is i-1
                address1 = routeStops[options[i-1][0]][4]
                address2 = request[0]
                deltaTime =+self.mapper.getRouteInfo(address1, address2)['time']
                
            # if ride exists after drop-off
            if options[i][2] < len(routeStops):
                # compute and add distance after drop-off. pushes existing i forward, so next ride is i
                address1 = routeStops[options[i][2]][4]
                address2 = request[0]
                deltaTime =+self.mapper.getRouteInfo(address1, address2)['time']
                
            if options[i][0] == options[i][2]:
                # compute and add distance between pick-up and drop-off
                address1 = request[0]
                address2 = request[1]
                deltaTime =+self.mapper.getRouteInfo(address1, address2)['time']
                
            # if not same insert slot,meening they are next to each other, then rides exist between
            else:
                # compute and add distance after pick-up and next ride in route
                address1 = request[0]
                address2 = routeStops[options[i][0]][4]
                deltaTime =+self.mapper.getRouteInfo(address1, address2)['time']
                # compute and add distance before drop-off and previous ride in route
                address1 = routeStops[options[i][0] - 1 ][4]
                address2 = request[1] 
                deltaTime =+self.mapper.getRouteInfo(address1, address2)['time']

            options[i].append(deltaTime)
            deltaRequestTime = abs(options[i][1] - request[2])
            options[i].append(deltaRequestTime)
        
        # sort by reverse importance, lowest drop off index, deltaTime from request, distance
        options = sorted(options, key=lambda x:x[2])
        options = sorted(options, key=lambda x:x[5])
        options = sorted(options, key=lambda x:x[4])
        print()
        print("284 - vehicle drive time test ")
        print(options)
        print()
        
        # remove sorting columns
        options = [sublist[:4] for sublist in options]
        
        return options
    
    
    # Author(s):  Richard Kershner
    # Description:
    #        Sorts options based on vectors of passangers - vectors of request
    #            (requestX2 - passangerSumX2 - (requestX1 - passangerSumX1)
    #          + (requestY2 - passangerSumY2 - (requestY1 - passangerSumY1)
    #        Multiple equal options sort on time closest to requested time
    def get_sorted_passangerXY_reqXY(self, routeStops, request):
        
        # retrieve options available
        options = self.get_options(request[2], stops)
        
        # although this is clunky, it works.
        #  X1 and Y1 are the first value in the Xs and Ys while X2 and Y2 are the last
        rMap = mapper.getRouteInfo(request[0], request[1])
        
        
        # append difference in pick-up option vs request and default weight 0.0
        for i in range(len(options)):
            options[i].append(abs(request[2]-options[i][1]))
            options[i].append(0.0)
        
        rX1 = rMap['Xs'][0]
        rY1 = rMap['Ys'][0]
        rX2 = rMap['Xs'][-1]
        rY2 = rMap ['Ys'][-1]
        
        # recreate request list
        passangerDict = dict()
        for stop in routeStops:
            # looking for ID: [X1, Y1, X2, Y2]
            if stop[0] not in passangerDict:
                # address1 = stop[4]
                #stopMap = mapper.getRouteInfo(stop[4])
                passangerDict[stop[0]] = request[0] # adderss 1 place holder
            else:
                address1 = passangerDict[stop[0]] # place holder of address 1
                address2 = request[1]
                rMap = mapper.getRouteInfo(address1, address2)
                
                # change dictionar to ID = X1, Y1, X2, Y2
                passangerDict[stop[0]] = [rMap['Xs'][0], rMap['Ys'][0], rMap['Xs'][-1], rMap['Ys'][-1]]
            
        passangers = []
        
        # go through stops and update options as they occur
        #    This allows for the passangers array to update and track who is on the vehicle
        for i in range(len(routeStops)):
            
            # process insert before boarding and unboarding passangers
            #    insert always goes before next stop
            for j in range(len(options)):
                if i == j:
                    # rX1, rY1, rX2, rY2
                    # average passanger location pX1, pY1, pX2, pY2.  Default
                    pXY = [0.0, 0.0, 0.0, 0.0]
                    weight = 0.0 # default, if no rides, should zero out
                    
                    
                    if len(passangers) != 0: 
                        for p in passangers: 

                            # same order, X1, Y1, X2, Y2.. so k is 0 to 3
                            for k in range(4):
                                pXY[k] =+ passangerDict[options[j][0]][k]
                            # average values

                            for k in range(4):
                                pXY[k] = passangerDict[options[j][0]][k] / len(passangers)
                                
                        # add weight
                        #            (requestX2 - passangerSumX2) - (requestX1 - passangerSumX1)
                        #          + (requestY2 - passangerSumY2) - (requestY1 - passangerSumY1)               
                        weight = (rX2 - pXY[2]) - (rX1 - pXY[0])
                        weight =+ (rY2 - pXY[3]) - (rY1 - pXY[1])
                    options[i].append(weight)
            
            # pickup passanger if not onboard
            if routeStops[i][0] not in passangers:
                passangers.append(routeStops[i][0])
                
            # dropoff passanger if already on board
            else:
                passangers = list(filter(lambda p: p!= routeStops[i][0], passangers))
        
        # sort by reverse importance, lowest drop off index, deltaTime from request, weighted
        options = sorted(options, key=lambda x:x[2])
        options = sorted(options, key=lambda x:x[4])
        options = sorted(options, key=lambda x:x[5])
        print()
        print("278 - drive time test RAW")
        print(options)
        print()
        
        # remove sorting columns
        options = [sublist[:4] for sublist in options]
        
        return options
    
    # ================= END ALGORITHMS ================
        
        
    def get_algorithms(self):
        algorithmDict = dict()
        for method in dir(self):
            if method.startswith('get_sorted') is True:
                algorithmDict[method] = getattr(self, method)
        return algorithmDict
    
    def get_dropoffOptions(self, pickupOptions, stops):
        # get options. Route must be adjusted first and tested so as to not
        #    have dead drive times ???? What does that look like??????
        
        options = []
        
        # --- loop through pick-up options
        #       append pickup point, pickup time, dropoff pointer, dropoff time.
        #       One row for each different dropoff option
        for option in pickupOptions:
            pnt_PU = option[0]
            time_PU = option[1]
            
            # --- add option directly after pickup
            #     note, in scheduling route, drop off is scheduled first so this equals pickup
            options.append([pnt_PU, time_PU, pnt_PU, time_PU])
            
            # --- for each pickup option, loop through dropoff options
            #         Only if inside max ride time
            #    note: anything before is already tested, so always +1 on pointer
            for pnt_DO in range(pnt_PU + 1, len(stops)):
                if stops[pnt_DO][1] - time_PU <= self.MAX_RIDE_TIME:
                    newOption = [pnt_PU, time_PU, pnt_DO +1, stops[pnt_DO][1]]
                    # --- only add if not a duplicate
                    if newOption not in options:
                        options.append(newOption)
                
        return options
    
    # Author:  Richard Kershner
    # Description: Supplies final options list for list.
    # Requires:
    #    requestTime : from request of [adderss1, address2, time]
    #    stops: from route stops
    # Uses: get_pickupOpstions() and get_dropoffOptions()
    def get_options(self, requestTime, stops):
        
        if stops == [[]]:
            return [[0,requestTime, 0, requestTime + 1]]
        
        optionsPU = self.get_pickupOptions(requestTime, stops)
        
        return self.get_dropoffOptions(optionsPU, stops)
    
    
    # Author:  Richard Kershner
    # Description: Supplied with existing stops, returns pick_options
    #        1.  Each stop generates an option before and after
    #        2.  Requests only generated if time is with in minimum and maximium pick_up window
    # Requires:
    #    requestTime : from request of [adderss1, address2, time]
    def get_pickupOptions(self, requestTime, stops):
        # pickup, dropoff... options.
        # pointer, time, pointer, time
        # first is [requested slot, requested time, requested slot, requested time]
        puOptions = []
        
        pnt = 0
        
        for s in range(len(stops)):
            
            stopTime = stops[s][1]
            
            # --- test valid pickup window
            if (stopTime > requestTime + self.MIN_SCHEDULE_WINDOW) and (stopTime < requestTime + self.MAX_SCHEDULE_WINDOW):
                # insert right before and after stop.  Time should match stops time
                #        Time will be pushed back or forward during testing if option fits by Route class
                puOptions.append([s, stopTime])
                puOptions.append([s+1, stopTime])
                
            # --- used to make sure actual requested time is an option for pickup
            if stopTime <= requestTime:
                pnt = pnt + 1
                
        # --- add actual request time to puOptions if it is not already in there
        if [pnt, requestTime] not in puOptions:
            puOptions.append([pnt, requestTime])
        
        return puOptions


# =================== inline Test ====================
import sys
sys.path.append(r'D:\programming\school\Capstone\modules')
sys.path.append(r'...\modules')
from Mapper import Mapper

def printStops(stops):
    print()
    print("stops for testing")
    for i in range(len(stops)):
        print(i, "  --> ",stops[i])
        
if __name__ == "__main__":
    mapper = Mapper()
    algorithms = Algorithms(mapper)
    
    
    stops = [[3, -80, -80, -6, '2114 Lincoln St']]
    stops.append([1, 31, 0, 10, '879 Neon Forest Circle'])
    stops.append([1, 33, 5, 33, '1819 Sunlight Dr'])
    stops.append([2, 35, 30, 45, '1715 Iron Horse Dr'])
    stops.append([3, 45, 45, 60, '2114 Lincoln St'])
    stops.append([2, 80, 80, 90, '1819 Sunlight Dr'])
    printStops(stops)
    print()
    
    request = ['1225 Ken Pratt Blvd', '225 E 8th Ave', 35]
    requestTime = request[2]
    print("request: ", request)
    print()
    
    pickupOptions = algorithms.get_pickupOptions(requestTime, stops)
    print("pickup options: ", pickupOptions)
    print()
    
    options = algorithms.get_dropoffOptions(pickupOptions, stops)
    print("full options")
    for option in options: print(option)
    print()
        
    options = algorithms.get_options(requestTime, stops)
    for option in options: print(option)
    print()
        
    # --- algorithms
    #        all algorithms require: routeStops, request
    #            This keeps them uniform in testing
    print()
    print("list algorithms")
    algorithmDict = algorithms.get_algorithms() 
    
    # routeStops, request
    
    
    for key, item in algorithmDict.items():
        options = item(stops, request)
        print(key, options)

        
    print("--- DONE ---")
    
    


stops for testing
0   -->  [3, -80, -80, -6, '2114 Lincoln St']
1   -->  [1, 31, 0, 10, '879 Neon Forest Circle']
2   -->  [1, 33, 5, 33, '1819 Sunlight Dr']
3   -->  [2, 35, 30, 45, '1715 Iron Horse Dr']
4   -->  [3, 45, 45, 60, '2114 Lincoln St']
5   -->  [2, 80, 80, 90, '1819 Sunlight Dr']

request:  ['1225 Ken Pratt Blvd', '225 E 8th Ave', 35]

pickup options:  [[1, 31], [2, 31], [2, 33], [3, 33], [3, 35], [4, 35], [4, 45], [5, 45]]

full options
[1, 31, 1, 31]
[1, 31, 3, 33]
[1, 31, 4, 35]
[1, 31, 5, 45]
[2, 31, 2, 31]
[2, 31, 4, 35]
[2, 31, 5, 45]
[2, 33, 2, 33]
[2, 33, 4, 35]
[2, 33, 5, 45]
[3, 33, 3, 33]
[3, 33, 5, 45]
[3, 35, 3, 35]
[3, 35, 5, 45]
[3, 35, 6, 80]
[4, 35, 4, 35]
[4, 35, 6, 80]
[4, 45, 4, 45]
[4, 45, 6, 80]
[5, 45, 5, 45]

[1, 31, 1, 31]
[1, 31, 3, 33]
[1, 31, 4, 35]
[1, 31, 5, 45]
[2, 31, 2, 31]
[2, 31, 4, 35]
[2, 31, 5, 45]
[2, 33, 2, 33]
[2, 33, 4, 35]
[2, 33, 5, 45]
[3, 33, 3, 33]
[3, 33, 5, 45]
[3, 35, 3, 35]
[3, 35, 5, 45]
[3, 35, 6, 80]
[4, 35, 4, 35]
[4,